# Data Cleanning Notebook

In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import textblob
import pickle
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer 

nltk.download("words")
from nltk.corpus import words
pd.set_option('display.max_columns', 10000)

[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [2]:
df=pd.read_csv('judge-1377884607_tweet_product_company.csv',encoding='unicode_escape')
df.index=list(df.index)

In [3]:
#resetting shape of Df and cleaning up target variable.
df.columns=['tweet','company','emotion']
df=df.drop([6]).reset_index(drop=True)
df['emotion'] = df['emotion'].apply(lambda x: "No emotion toward brand or product" if x == "I can't tell" else x)

In [5]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)

    return text

round1 = lambda x: clean_text_round1(x)

In [6]:
#cleaning tweet  wit regex.
df['clean_tweet']=df.tweet.apply(round1)

Creating a new column that interprets the tweet and extracts what company is the tweet talking about. It works by taking in the tweet, cleaning the words and then extracting whatever the tweet is referencing. If the tweet mentiones both brands, both brands will be taken in consideration in the appended column

In [7]:
punctuation=[",",".","#","!","@","$","?","$","%","&","-","_",":","=", "/",";",'(',")", "{","}","|"]
apple=['iPhone',"iphones",'i phones',"iPad&quot;","iphone ",'i Pad','i Pad',"Apple.","iPad?","#iPad","#iPhone",'iPad','Mac',"iphone,",'iPod','ipod','ipad','mac','Apple','apple','Apple\'s','apple\'s',"#Apple",'#apple','I-Pad']
google=['Google','google','android','Android','Google.','android,',"#android","Google's",'google\'s','googles' ,'#google','#Google']

In [8]:
for index in df.index:
    tweet="".join(i if i not in punctuation else " " for i in df.tweet[index])
    condition=0
    df.tweet[index]=tweet

    
    if bool(set(tweet.split())&set(apple))==True:
        df.company[index]=(list(set(tweet.split())&set(apple))[0].title().strip("'s")).strip('\'S')
        condition=1
    else:
        condition=0
        
    if (bool(set(tweet.split())&set(google))==True) and condition==0:
        df.company[index]=(list(set(tweet.split())&set(google))[0].title().strip('\'s')).strip('\'S')
        
    elif (bool(set(tweet.split())&set(google))==True) and condition==1:
        df.company[index]+= " & "+(list(set(tweet.split())&set(google))[0].title().strip('\'s')).strip('\'S')
df['company'] = df['company'].apply(lambda x: "iPad" if x == "Ipad" else x)
df['company'] = df['company'].apply(lambda x: "iPhone" if x == "Iphone" else x)

In [9]:
#Creating a new column with each product main brand
df['brand'] = df['company'].apply(lambda x: 'Apple' if x in apple else 'Google' if x in google else '')

In [10]:
#cleaned DF
df

,tweet,company,emotion,clean_tweet,brand
0,wesley83 I have a 3G iPhone After 3 hrs twe...,iPhone,Negative emotion,i have a iphone after hrs tweeting at...,Apple
1,jessedee Know about fludapp Awesome iPad i...,iPad,Positive emotion,jessedee know about fludapp awesome ipad i...,Apple
2,swonderlin Can not wait for iPad 2 also The...,iPad,Positive emotion,swonderlin can not wait for ipad also the...,Apple
3,sxsw I hope this year's festival isn't as cra...,iPhone,Negative emotion,sxsw i hope this year s festival isn t as cra...,Apple
4,sxtxstate great stuff on Fri SXSW Marissa M...,Google,Positive emotion,sxtxstate great stuff on fri sxsw marissa m...,Google
...,...,...,...,...,...
9087,Ipad everywhere SXSW link,iPad,Positive emotion,ipad everywhere sxsw link,Apple
9088,Wave buzz RT mention We interrupt your re...,Google,No emotion toward brand or product,wave buzz rt mention we interrupt your re...,Google
9089,Google's Zeiger a physician never reported po...,Google,No emotion toward brand or product,google s zeiger a physician never reported po...,Google
9090,Some Verizon iPhone customers complained their...,iPhone,No emotion toward brand or product,some verizon iphone customers complained their...,Apple


In [11]:
#this function creates a lemmatized list for each value provided.
lemmatizer = WordNetLemmatizer() 
def lemmer(text):
    lemmatizer = WordNetLemmatizer() 
    filtered_review=text.split()
    lemmatized_review=[]
    for w in filtered_review:
        lemmatized_review.append(lemmatizer.lemmatize(w))

    return lemmatized_review


lemming = lambda x: lemmer(x)

In [12]:
#Lemming each tweet of the dataframe
df['lemming_tweet']=df.clean_tweet.apply(lemming)

This for loop iterates trough each tweet and appends it to a dictionary that corresponds to each company. This
is useful for the sentiment analisist that will be thode further. 

In [34]:

%%time
dictionary={}
companies=list(df.company.unique())
iteration=1
for company in companies:
    print("This is iteration: "+str(iteration)+" of "+str(len(companies)))
    tweet=[]
    for index in df[df["company"]==company].index:
        print('\r' +str(index), end="", flush=True)
        tweet.append(" ".join(i if i  in words.words() else "" for i in df.tweet[index].lower().split()))
    dictionary[company]=tweet
    print('\n')
    iteration+=1
dictionary

This is iteration: 1 of 18
9090

This is iteration: 2 of 18
9087

This is iteration: 3 of 18
9091

This is iteration: 4 of 18
9086

This is iteration: 5 of 18
9081

This is iteration: 6 of 18
9037

This is iteration: 7 of 18
8538

This is iteration: 8 of 18


This is iteration: 9 of 18
8817

This is iteration: 10 of 18
8983

This is iteration: 11 of 18
8616

This is iteration: 12 of 18
8705

This is iteration: 13 of 18
5263

This is iteration: 14 of 18
8827

This is iteration: 15 of 18
8752

This is iteration: 16 of 18
6163

This is iteration: 17 of 18
5390

This is iteration: 18 of 18
7973

CPU times: user 3h 13min 23s, sys: 28min 49s, total: 3h 42min 13s
Wall time: 3h 46min 33s


{'iPhone': [' i have a   after    at rise  it was dead i need to upgrade   at ',
  ' i hope this  festival  as  as this    ',
  'i just   is coming this weekend how many   will be an hour late at  come  morning  ',
  'just added my   to  matching people on   also  the    nicely done',
  'photo just  the    which is really nice    ',
  'hey  how long do you think it  us to make an  case answer mention   and  make you one',
  'for i pad  mention new  for  now in the  store   to   by link',
  'hand      with a     link',
  'i love my mention  case from  but i  get my phone out of it fail',
  'new post mention    it easy to connect on all social  with people you meet link ',
  'nice that mention   is  today  yesterday were ridiculous ',
  'what mention  does not provide     my mind about going next year',
  '  mention new  for  now in the  store   to   by  link',
  'know that quot  quot  to quot satanic quot on an   just  ',
  'funny how  is  but not  only a matter of  at this point at lea

In [14]:
# dictionary = pickle.load( open( "dictionary.pickle", "rb" ) )

In [35]:
#Transforming the dictionary into a DF and 
sentiment_df=pd.DataFrame(columns=['company','tweets'])
sentiment_df['tweets']=list(dictionary.values())
sentiment_df['company']=list(dictionary.keys())

In [36]:
#Exporting the 2 clean databases for modeding.
sentiment_df.to_csv('Sentiment.csv')
df.to_csv('clean_df.csv')